## Create Dataset

In [ ]:
import nltk
import random

In [ ]:
def load_data(path):
  with open(path, 'r', encoding ='utf-8') as file:
    sentences = [line.strip()for line in file.readlines()]
  return sentences

shake = load_data('/content/drive/MyDrive/cs224n/final project/shakespeare.nltktok')
modern = load_data('/content/drive/MyDrive/cs224n/final project/normal.nltktok')

print(sorted(shake, key=len, reverse=True))


labeled_data = [(sentence, 1) for sentence in shake] + [(sentence, 0) for sentence in modern]

random.shuffle(labeled_data)

texts = []
labels = []


for tup in labeled_data:
  texts.append(tup[0])
  labels.append(tup[1])

max_length = max(len(t) for t in texts)

print(max_length)
print(texts[0])
print(labels[0])






In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from torch.utils.data import Dataset
import pickle
import random
import numpy as np
import torch
import argparse
import json
from tqdm import tqdm
from transformers import BertTokenizer
import torch.nn.functional as F


class ShakespeareDataset(Dataset):
     def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.max_length = max_length
        self.tokenizer = tokenizer

     def __len__(self):
        # returns the length of the dataset
        return len(self.texts)

     def __getitem__(self, idx):
        text, label = self.texts[idx], self.labels[idx]
       # print(text)
       # print(label)
        encoding = self.tokenizer(text, return_tensors='pt', max_length=256, padding='max_length', truncation=True)
       # print(encoding['input_ids'].squeeze().shape)
       # print(encoding['attention_mask'].squeeze().shape)
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {'input_ids':input_ids, 'attention_mask':attention_mask, 'label':torch.tensor(label)}


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bruh = ShakespeareDataset(texts, labels, max_length, tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

##Train

##Attempt 2

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

model = BertForSequenceClassification.from_pretrained("bert-base-uncased")



train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

train_dataset = ShakespeareDataset(train_texts, train_labels, max_length=256, tokenizer=tokenizer)
val_dataset = ShakespeareDataset(val_texts, val_labels, max_length=256, tokenizer=tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(5):
    model.train()
    total_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        print(f"Iteration Loss: {loss.item():.4f}")


    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Average Loss: {avg_loss:.4f}")


In [1]:
model.save_pretrained("/content/drive/MyDrive/cs224n")


NameError: name 'model' is not defined

In [ ]:
from sklearn.metrics import accuracy_score, classification_report


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

evaluate(model, val_loader, device)

#torch.save(model.state_dict(), "'/content/drive/MyDrive/cs224n/final project/")